In [3]:
import ee # Google Earth Engine
import datetime
#import ipyleaflet
import numpy as np
import matplotlib.pyplot as plt
import folium

Initialize Google Earth Engine (GEE)

In [4]:
ee.Initialize()

Specify an area of interest

In [39]:
# area of interest has lat, lon of (-10.883689, -44.005800)
Brazil_ex1_lat, Brazil_ex1_lon = -10.883689, -44.005800
Brazil_ex1_edge_len = 0.2

In [46]:
US_IL_lat, US_IL_lon = 40.707570, -88.804750
US_IL_edge_len = 0.2

In [79]:
def satallite_imagery(source, center_lat, center_lon, edge_len,
                      start_date, end_date, 
                      plot_option, time_series):
    # Sentinel-2 Level 1-C: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2
    # Sentinel-2 Level 2-A: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR
    # Landsat Tier structure: https://developers.google.com/earth-engine/landsat#landsat-collection-structure
    if source == "Sentinel2_1C":
        source_loc = 'COPERNICUS/S2'
    elif source == "Sentinel2_2A":
        source_loc = 'COPERNICUS/S2_SR'
    elif source == "Landsat7":
        source_loc = 'LANDSAT/LE07/C01/T1'
    elif source == "Landsat8":
        source_loc = 'LANDSAT/LC08/C01/T1_SR'
    else:
        raise Exception('Invalid source of satellite imagery')
        
    area_of_interest = ee.Geometry.Polygon([[center_lon-edge_len/2, center_lat-edge_len/2], 
                           [center_lon-edge_len/2, center_lat+edge_len/2], 
                           [center_lon+edge_len/2, center_lat+edge_len/2], 
                           [center_lon+edge_len/2, center_lat-edge_len/2]])
    
    img_collect = (ee.ImageCollection(source_loc)
                 .filterDate(start_date, end_date)
                 .filterBounds(area_of_interest))
    
    if 'Sentinel' in source:
        img_collect = img_collect.filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",10)
    elif 'Landsat' in source:
        img_collect = img_collect.filter(ee.Filter.lt('CLOUD_COVER', 3))
    
    assert (img_collect.size().getInfo()>0), "No valid image"
    
    print("Number of images in the collection: ", img_collect.size().getInfo())
    
    if 'Sentinel' in source:
        # Reference: https://www.satimagingcorp.com/satellite-sensors/other-satellite-sensors/sentinel-2a/
        band_blue = 'B2' #10m
        band_green = 'B3' #10m
        band_red = "B4"  #10m
        band_nir = 'B8'  #10m
    # Reference: https://landsat.gsfc.nasa.gov/landsat-data-continuity-mission/
    elif 'Landsat7' in source:
        # Reference: https://www.usgs.gov/land-resources/nli/landsat/landsat-7?qt-science_support_page_related_con=0#qt-science_support_page_related_con
        band_blue = 'B1' #30m
        band_green = 'B2' #30m
        band_red = "B3"  #30m
        band_nir = 'B4'  #30m
    elif 'Landsat8' in source:
        # Reference: https://www.usgs.gov/faqs/what-are-best-landsat-spectral-bands-use-my-research?qt-news_science_products=0#qt-news_science_products
        band_blue = 'B2' #30m
        band_green = 'B3' #30m
        band_red = "B4"  #30m
        band_nir = 'B5'  #30m    
    
    def calc_NDVI(img):
        ndvi = ee.Image(img.normalizedDifference([band_nir, band_red])).rename(["ndvi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(ndvi)
        return composite
    
    def calc_YYYYMM(img):
        return img.set('YYYYMM', img.date().format("YYYYMM"))
    
    def add_ee_layer(self, ee_object, vis_params, name):
        try:    
            if isinstance(ee_object, ee.image.Image):    
                map_id_dict = ee.Image(ee_object).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                tiles = map_id_dict['tile_fetcher'].url_format,
                attr = 'Google Earth Engine',
                name = name,
                overlay = True,
                control = True
                ).add_to(self)
            elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
                ee_object_new = ee_object.mosaic()
                map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                tiles = map_id_dict['tile_fetcher'].url_format,
                attr = 'Google Earth Engine',
                name = name,
                overlay = True,
                control = True
                ).add_to(self)
            elif isinstance(ee_object, ee.geometry.Geometry):    
                folium.GeoJson(
                data = ee_object.getInfo(),
                name = name,
                overlay = True,
                control = True
            ).add_to(self)
            elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
                ee_object_new = ee.Image().paint(ee_object, 0, 2)
                map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                tiles = map_id_dict['tile_fetcher'].url_format,
                attr = 'Google Earth Engine',
                name = name,
                overlay = True,
                control = True
            ).add_to(self)

        except:
            print("Could not display {}".format(name))

    # Add EE drawing method to folium.
    folium.Map.add_ee_layer = add_ee_layer
    
    img_collect_calc = img_collect.map(calc_YYYYMM).map(calc_NDVI)
    
    unique_month = list(set([item['properties']['YYYYMM'] for item in img_collect_calc.getInfo()['features']]))
    unique_month.sort()
    
    subset1_collection = img_collect_calc.filter(ee.Filter.eq('YYYYMM','201801'))
    img_display = img_collect_calc.median()
    
    
    # Create a folium map object.
    myMap = folium.Map(location=[center_lat, center_lon], zoom_start=8)
    
    if plot_option == 'RGB':
        if 'Sentinel' in source:
            visParams = {'min':0, 'max':3000}
        elif 'Landsat' in source:
            visParams = {'min':0, 'max':255}
        if time_series=='no':
            myMap.add_ee_layer(img_collect_calc.median().select(band_red, band_green, band_blue), visParams, name=source+' '+plot_option)
        elif time_series=="monthly":
            for month in unique_month:
                myMap.add_ee_layer(img_collect_calc.filter(ee.Filter.eq('YYYYMM',month)).median().select(band_red, band_green, band_blue), visParams, name=source+' '+plot_option+' '+month)
    elif plot_option == 'VI':
        visParams = {'min':0, 'max':1, 'palette': ['red', 'yellow', 'green']}
        if time_series=='no':
            myMap.add_ee_layer(img_collect_calc.median().select("ndvi"), visParams, name=source+' '+plot_option)
        elif time_series=="monthly":
            for month in unique_month:
                myMap.add_ee_layer(img_collect_calc.filter(ee.Filter.eq('YYYYMM',month)).median().select("ndvi"), visParams, name=source+' '+plot_option+' '+month)

    # Add a layer control panel to the map.
    myMap.add_child(folium.LayerControl())
    
    return myMap

In [80]:
satallite_imagery(source="Landsat7", center_lat=Brazil_ex1_lat, center_lon=Brazil_ex1_lon, edge_len=Brazil_ex1_edge_len,
                  start_date='2018-05-01', end_date='2018-7-31',
                  plot_option='RGB', time_series="no")

Number of images in the collection:  6


In [81]:
satallite_imagery(source="Sentinel2_1C", center_lat=Brazil_ex1_lat, center_lon=Brazil_ex1_lon, edge_len=Brazil_ex1_edge_len,
                  start_date='2018-05-01', end_date='2018-7-31',
                  plot_option='RGB', time_series="no")

Number of images in the collection:  124


In [83]:
satallite_imagery(source="Sentinel2_1C", center_lat=Brazil_ex1_lat, center_lon=Brazil_ex1_lon, edge_len=Brazil_ex1_edge_len,
                  start_date='2018-05-01', end_date='2018-7-31',
                  plot_option='VI', time_series="no")

Number of images in the collection:  124


In [84]:
satallite_imagery(source="Sentinel2_1C", center_lat=US_IL_lat, center_lon=US_IL_lon, edge_len=US_IL_edge_len,
                  start_date='2018-05-01', end_date='2018-7-31',
                  plot_option='VI', time_series="no")

Number of images in the collection:  28


In [86]:
satallite_imagery(source="Sentinel2_1C", center_lat=US_IL_lat, center_lon=US_IL_lon, edge_len=US_IL_edge_len,
                  start_date='2018-1-01', end_date='2018-12-31',
                  plot_option='VI', time_series="monthly")

Number of images in the collection:  88


In [87]:
satallite_imagery(source="Sentinel2_1C", center_lat=US_IL_lat, center_lon=US_IL_lon, edge_len=US_IL_edge_len,
                  start_date='2018-1-01', end_date='2018-12-31',
                  plot_option='VI', time_series="no")

Number of images in the collection:  88
